In [50]:
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 30


In [16]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf   

# Creating the teams dataframe

In [17]:
TEAMNAMES=pd.read_csv('../Data/from_sqlite_Data/Team.csv')
LEAGUENAMES= pd.read_csv('../Data/from_sqlite_Data/League.csv')
df2 = pd.read_csv('../Data/from_sqlite_Data/bigfinal.csv')
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
df2.drop(['league_id'],axis=1,inplace=True)
df2.drop(['id'],axis=1,inplace=True)

In [18]:
df2.head(n=1)

,country_id,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,Belgium Jupiler League,2008/2009,2008-08-17 00:00:00,KRC Genk,Beerschot AC,1,1,1.73,3.4,5,1.75,3.35,4.2,1.85,3.2,3.5,1.8,3.3,3.75,<null>,<null>,<null>,1.7,3.3,4.33,1.9,3.3,4,1.65,3.4,4.5,1.78,3.25,4,1.73,3.4,4.2


In [19]:
season = df2.iloc[:,:7]
FTR=[]
for i in range(len(season)):
    if season['FTHG'][i]>season['FTAG'][i]:
        FTR.append('H')
    if season['FTHG'][i]<season['FTAG'][i]:
        FTR.append('A')
    if season['FTHG'][i]==season['FTAG'][i]:
        FTR.append('D')
    
season['FTR']=FTR   

In [20]:
season=season.sort('Date')

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [21]:
season.reset_index(inplace=True)


In [22]:
season.drop(['index'],axis=1,inplace=True)

In [60]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

## Allteamsresults is a list of dataframes and each dataframe represents a specific team. To access info on a team use team index. The position of the team in the list "teams" gives  the index of the team in the "allteamsresults" list of dataframes.

## Now, lets calculate some other features

## 1) Goal scored and received - home and away.
### We will shift of one match back each calculation, which means, each row will have the present match and the statistics goals scored up to that day (before the match).

In [57]:
def calculate_for_team_i(ALL_TEAM,team_lists,team_index):
    
    #create the lists for goals 
    Goal_scored_home=[]
    Goal_received_home=[]
    Goal_scored_away=[]
    Goal_received_away=[]
    
    #total_goal
    total_goal_scored=[]
    total_goal_received=[]
    
    #create the list for averages goals
    Av_Goal_scored_home=[]
    Av_Goal_received_home=[]
   

    Av_Goal_scored_away=[]
    Av_Goal_received_away=[]
       
    #create the list for averages total goals
    Av_total_goal_scored=[]
    Av_total_goal_received=[]
    
    #create the lists for points
    Point_home=[]
    Point_away=[]

    total_point=[]

    #create the list for average Points
    Av_Point_home=[]
    Av_Point_away=[]

    Av_total_point=[]
    
    #ROUND_INDEX
    round_index=[]

    
    k=1
    
    Goal_scored_home.append(0)
    Goal_received_home.append(0)
    Goal_scored_away.append(0)
    Goal_received_away.append(0)
    total_goal_scored.append(0)
    total_goal_received.append(0)
    Av_Goal_scored_home.append(0)
    Av_Goal_received_home.append(0)
    Av_Goal_scored_away.append(0)
    Av_Goal_received_away.append(0)
    Av_total_goal_scored.append(0)
    Av_total_goal_received.append(0)
    Point_home.append(0)
    Point_away.append(0)
    total_point.append(0)
    Av_Point_home.append(0)
    Av_Point_away.append(0)
    Av_total_point.append(0)
    
    for h in allteamsresults[team_index].df['season'].unique():
    
        seasonal_index=allteamsresults[team_index].df[allteamsresults[team_index].df['season']==h].index
        Home=0
        Away=0
        total=0
        round_in=1
        
        for i in seasonal_index:
            
            round_index.append(round_in)
            round_in=round_in+1
            
            #HOME
            if ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_home.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_scored_home[k-1])
                Goal_received_home.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_received_home[k-1])  
                Goal_scored_away.append(Goal_scored_away[k-1])
                Goal_received_away.append(Goal_received_away[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTHG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTAG'][i])

                #Goal average
                Home=Home+1
                total=Home+Away
                Av_Goal_scored_home.append(float(Goal_scored_home[k])/Home)
                Av_Goal_received_home.append(float(Goal_received_home[k])/Home)
                Av_Goal_scored_away.append(Av_Goal_scored_away[k-1])
                Av_Goal_received_away.append(Av_Goal_received_away[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_home.append(3+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_home.append(1+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_home.append(0+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)   
                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                Point_away.append(Point_away[k-1])
                Av_Point_away.append(Av_Point_away[k-1])

            #AWAY
            if ALL_TEAM[team_index].df['HomeTeam'][i]!=team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_away.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_scored_away[k-1])
                Goal_received_away.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_received_away[k-1])
                Goal_scored_home.append(Goal_scored_home[k-1])
                Goal_received_home.append(Goal_received_home[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTAG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTHG'][i])

                #Goal average
                Away=Away+1
                total=Home+Away

                Av_Goal_scored_away.append(float(Goal_scored_away[k])/Away)
                Av_Goal_received_away.append(float(Goal_received_away[k])/Away)
                Av_Goal_scored_home.append(Av_Goal_scored_home[k-1])
                Av_Goal_received_home.append(Av_Goal_received_home[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_away.append(3+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_away.append(1+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_away.append(0+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)


                Point_home.append(Point_home[k-1])
                Av_Point_home.append(Av_Point_home[k-1])


            k=k+1

        
        Goal_scored_home[-1]=0
        Goal_received_home[-1]=0
        Goal_scored_away[-1]=0
        Goal_received_away[-1]=0
        total_goal_scored[-1]=0
        total_goal_received[-1]=0


        Av_Goal_scored_home[-1]=0
        Av_Goal_received_home[-1]=0
        Av_Goal_scored_away[-1]=0
        Av_Goal_received_away[-1]=0
        Av_total_goal_scored[-1]=0
        Av_total_goal_received[-1]=0


        Point_home[-1]=0
        Point_away[-1]=0
        Av_Point_home[-1]=0
        Av_Point_away[-1]=0

        total_point[-1]=0
        Av_total_point[-1]=0
        

    del Goal_scored_home[-1]
    del Goal_received_home[-1]
    del Goal_scored_away[-1]
    del Goal_received_away[-1]
    del total_goal_scored[-1]
    del total_goal_received[-1]


    del Av_Goal_scored_home[-1]
    del Av_Goal_received_home[-1]
    del Av_Goal_scored_away[-1]
    del Av_Goal_received_away[-1]
    del Av_total_goal_scored[-1]
    del Av_total_goal_received[-1]


    del Point_home[-1]
    del Point_away[-1]
    del Av_Point_home[-1]
    del Av_Point_away[-1]

    del total_point[-1]
    del Av_total_point[-1]
    
    ALL_TEAM[team_index].df['Round']=round_index
    
    ALL_TEAM[team_index].df['GS_H']=Goal_scored_home
    ALL_TEAM[team_index].df['GS_A']=Goal_scored_away
    ALL_TEAM[team_index].df['GR_H']=Goal_received_home
    ALL_TEAM[team_index].df['GR_A']=Goal_received_away
    ALL_TEAM[team_index].df['TGS']=total_goal_scored
    ALL_TEAM[team_index].df['TGR']=total_goal_received
    
    
    ALL_TEAM[team_index].df['AGS_H']=Av_Goal_scored_home
    ALL_TEAM[team_index].df['AGS_A']=Av_Goal_scored_away
    ALL_TEAM[team_index].df['AGR_H']=Av_Goal_received_home
    ALL_TEAM[team_index].df['AGR_A']=Av_Goal_received_away
    ALL_TEAM[team_index].df['ATGS']=Av_total_goal_scored
    ALL_TEAM[team_index].df['ATGR']=Av_total_goal_received

    
    ALL_TEAM[team_index].df['Pnt_H']=Point_home
    ALL_TEAM[team_index].df['Pnt_A']=Point_away
    ALL_TEAM[team_index].df['TOTPT']=total_point

    
    ALL_TEAM[team_index].df['Av_Pnt_H']=Av_Point_home
    ALL_TEAM[team_index].df['Av_pnt_A']=Av_Point_away
    ALL_TEAM[team_index].df['Av_tot_pnt']=Av_total_point
    



    
    
    return ALL_TEAM[team_index].df


In [58]:
def calculate_past_match (ALL_TEAM,team_lists,team_index):
    
    T_L3M_Pnt=[]
    T_L5M_Pnt=[]
    
    H_L3M_Pnt=[]
    A_L3M_Pnt=[]
    
    H_L5M_Pnt=[]
    A_L5M_Pnt=[]
    
    
     
    for i in ALL_TEAM[team_index].df.index:
        
        
        #HOME
        if ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]:
            

            #Goal scored, received and total
            Goal_scored_home.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_scored_home[k-1])
            Goal_received_home.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_received_home[k-1])
    
    
    

In [62]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [63]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [64]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[8:]) + list(allteamsresults[away_index].df.loc[i].iloc[8:])
    tmp.loc[i]=tobeappend

In [65]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)

In [66]:
final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']


In [67]:
final

,country_id,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,H_ROUND,H_GS_H,H_GS_A,H_GR_H,H_GR_A,H_TGS,H_TGR,H_AvGS_H,H_AvGS_A,H_AvGR_H,H_AvGR_A,H_Av_TGS,H_Av_TGR,H_Pnt_H,H_Pnt_A,H_T_pnt,H_Av_Pnt_H,H_Av_pnt_A,H_Av_T_pnt,A_ROUND,Aw_GS_H,Aw_GS_A,Aw_GR_H,Aw_GR_A,Aw_TGS,Aw_TGR,Aw_AvGS_H,Aw_AvGS_A,Aw_AvGR_H,Aw_AvGR_A,Aw_Av_TGS,Aw_Av_TGR,Aw_Pnt_H,Aw_Pnt_A,Aw_T_pnt,Aw_Av_Pnt_H,Aw_Av_pnt_A,Aw_Av_T_pnt,DIFF_av_TP,DIFF_av_HP-AP,SUMM_av_HGSH+AGRA,SUMM_av_HGRH+AGSA,SUMM_av_HTGS+ATGR,SUMM_av_HTGR+ATGS
0,Switzerland Super League,2008/2009,2008-07-18 00:00:00,BSC Young Boys,FC Basel,1,2,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Switzerland Super League,2008/2009,2008-07-19 00:00:00,FC Aarau,FC Sion,3,1,H,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Switzerland Super League,2008/2009,2008-07-20 00:00:00,FC Luzern,FC Vaduz,1,2,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Switzerland Super League,2008/2009,2008-07-20 00:00:00,Neuchâtel Xamax,FC Zürich,1,2,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Switzerland Super League,2008/2009,2008-07-23 00:00:00,AC Bellinzona,Neuchâtel Xamax,1,2,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,1.0,0.0,2.0,0.0,1.0,2.0,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
5,Switzerland Super League,2008/2009,2008-07-23 00:00:00,FC Basel,Grasshopper Club Zürich,1,0,H,2.0,0.0,2.0,0.0,1.0,2.0,1.0,0.000000,2.000000,0.000000,1.000000,2.000000,1.000000,0.0,3.0,3.0,0.000000,3.000000,3.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,2.000000,1.000000
6,Switzerland Super League,2008/2009,2008-07-23 00:00:00,FC Zürich,FC Luzern,1,0,H,2.0,0.0,2.0,0.0,1.0,2.0,1.0,0.000000,2.000000,0.000000,1.000000,2.000000,1.000000,0.0,3.0,3.0,0.000000,3.000000,3.000000,2.0,1.0,0.0,2.0,0.0,1.0,2.0,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,4.000000,2.000000
7,Switzerland Super League,2008/2009,2008-07-24 00:00:00,FC Sion,BSC Young Boys,2,1,H,2.0,0.0,1.0,0.0,3.0,1.0,3.0,0.000000,1.000000,0.000000,3.000000,1.000000,3.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,1.0,0.0,2.0,0.0,1.0,2.0,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000
8,Switzerland Super League,2008/2009,2008-07-24 00:00:00,FC Vaduz,FC Aarau,0,2,A,2.0,0.0,2.0,0.0,1.0,2.0,1.0,0.000000,2.000000,0.000000,1.000000,2.000000,1.000000,0.0,3.0,3.0,0.000000,3.000000,3.000000,2.0,3.0,0.0,1.0,0.0,3.0,1.0,3.000000,0.000000,1.000000,0.000000,3.000000,1.000000,3.0,0.0,3.0,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000
9,Switzerland Super League,2008/2009,2008-07-26 00:00:00,FC Luzern,Neu

In [68]:
final.to_csv('../Data/from_sqlite_Data/realfinal.csv')

In [69]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')

In [70]:
final.drop(['Unnamed: 0'], axis=1,inplace=True)

In [71]:
England=final[final['country_id']=='England Premier League'].sort(columns='Date')
Belgium=final[final['country_id']=='Belgium Jupiler League'].sort(columns='Date')
France=final[final['country_id']=='France Ligue 1'].sort(columns='Date')
Germany=final[final['country_id']=='Germany 1. Bundesliga'].sort(columns='Date')
Italy=final[final['country_id']=='Italy Serie A'].sort(columns='Date')
Netherlands=final[final['country_id']=='Netherlands Eredivisie'].sort(columns='Date')
Poland=final[final['country_id']=='Poland Ekstraklasa'].sort(columns='Date')
Portugal=final[final['country_id']=='Portugal Liga ZON Sagres'].sort(columns='Date')
Scotland=final[final['country_id']=='Scotland Premier League'].sort(columns='Date')
Spain=final[final['country_id']=='Spain LIGA BBVA'].sort(columns='Date')
Swiss=final[final['country_id']=='Switzerland Super League'].sort(columns='Date')

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: Futur

In [72]:
England.reset_index(inplace=True)
Belgium.reset_index(inplace=True)
France.reset_index(inplace=True)
Germany.reset_index(inplace=True)
Italy.reset_index(inplace=True)
Netherlands.reset_index(inplace=True)
Poland.reset_index(inplace=True)
Portugal.reset_index(inplace=True)
Scotland.reset_index(inplace=True)
Spain.reset_index(inplace=True)
Swiss.reset_index(inplace=True)

In [73]:
England.drop(['index'],axis=1,inplace=True)
Belgium.drop(['index'],axis=1,inplace=True)
France.drop(['index'],axis=1,inplace=True)
Germany.drop(['index'],axis=1,inplace=True)
Italy.drop(['index'],axis=1,inplace=True)
Netherlands.drop(['index'],axis=1,inplace=True)
Poland.drop(['index'],axis=1,inplace=True)
Portugal.drop(['index'],axis=1,inplace=True)
Scotland.drop(['index'],axis=1,inplace=True)
Spain.drop(['index'],axis=1,inplace=True)
Swiss.drop(['index'],axis=1,inplace=True)